# Nye fylker 2024

Norway will be going to 15 fylker as of January 2024. This notebook is to update the mapping in muni_county_namedDialect_numericDialect_mapping_manual_additions_renamed.csv

Reference pages:

[Nye kommune og fylkesnummer](https://www.regjeringen.no/no/tema/kommuner-og-regioner/kommunestruktur/nye-kommune-og-fylkesnummer-fra-1.-januar-2024/id2924701/)

[Fylkesinndelingen fra 2024](https://www.regjeringen.no/no/tema/kommuner-og-regioner/kommunestruktur/fylkesinndelingen-fra-2024/id2922222/)

Viken deles i tre nye fylker: Østfold, Akershus og Buskerud.

Vestfold og Telemark deles i to nye fylker: Vestfold og Telemark.

Troms og Finnmark deles i to nye fylker: Troms og Finnmark.


In [32]:
new_fylke = [
    'Østfold',
    'Akershus',
    'Buskerud',
    'Vestfold',
    'Telemark',
    'Troms',
    'Finnmark'
]

In [2]:
import requests
from bs4 import BeautifulSoup

response = requests.get('https://www.regjeringen.no/no/tema/kommuner-og-regioner/kommunestruktur/nye-kommune-og-fylkesnummer-fra-1.-januar-2024/id2924701/')
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'xml')

In [33]:
fylke_tables = dict()

for button in soup.find_all('button'):
    button_text = button.text.split(' – ')[0].strip()
    if button_text in new_fylke:
        fylke_tables[button.get('aria-controls')]= button_text

fylke_tables


{'factbox2924704': 'Østfold',
 'factbox2924705': 'Akershus',
 'factbox2924706': 'Buskerud',
 'factbox2924707': 'Vestfold',
 'factbox2924708': 'Telemark',
 'factbox2924709': 'Troms',
 'factbox2924710': 'Finnmark'}

In [35]:
import pandas as pd
import io

kommune_to_new_fylke = {}

for div in soup.find_all('div'):
    div_id = div.get('id')
    if div_id in fylke_tables:
        table = div.find_all('table')[0]
        df = pd.read_html(io.StringIO(str(table)), flavor='bs4', header=0)[0]
        for kommune in list(df['Kommunenavn']):
            kommune_to_new_fylke[kommune] = fylke_tables[div_id]

kommune_to_new_fylke

{'Halden': 'Østfold',
 'Moss': 'Østfold',
 'Sarpsborg': 'Østfold',
 'Fredrikstad': 'Østfold',
 'Hvaler': 'Østfold',
 'Råde': 'Østfold',
 'Våler': 'Østfold',
 'Skiptvet': 'Østfold',
 'Indre Østfold': 'Østfold',
 'Rakkestad': 'Østfold',
 'Marker': 'Østfold',
 'Aremark': 'Østfold',
 'Bærum': 'Akershus',
 'Asker': 'Akershus',
 'Lillestrøm': 'Akershus',
 'Nordre Follo': 'Akershus',
 'Ullensaker': 'Akershus',
 'Nesodden': 'Akershus',
 'Frogn': 'Akershus',
 'Vestby': 'Akershus',
 'Ås': 'Akershus',
 'Enebakk': 'Akershus',
 'Lørenskog': 'Akershus',
 'Rælingen': 'Akershus',
 'Aurskog-Høland': 'Akershus',
 'Nes': 'Akershus',
 'Gjerdrum': 'Akershus',
 'Nittedal': 'Akershus',
 'Lunner': 'Akershus',
 'Jevnaker': 'Akershus',
 'Nannestad': 'Akershus',
 'Eidsvoll': 'Akershus',
 'Hurdal': 'Akershus',
 'Drammen': 'Buskerud',
 'Kongsberg': 'Buskerud',
 'Ringerike': 'Buskerud',
 'Hole': 'Buskerud',
 'Lier': 'Buskerud',
 'Øvre Eiker': 'Buskerud',
 'Modum': 'Buskerud',
 'Krødsherad': 'Buskerud',
 'Flå': 'Bus

Now update the CSV file

In [26]:
csv_df = pd.read_csv('/Users/plparson/Documents/development/DialectMapper/dialect_mapper/mapping_data/muni_county_namedDialect_numericDialect_mapping_manual_additions_renamed.csv')
csv_df.head()

,old_muni,new_muni,old_county,new_county,named_dialect,numeric_dialect
0,Risør,Risør,Aust-Agder,Agder,Sørlandsk,20
1,Grimstad,Grimstad,Aust-Agder,Agder,Sørlandsk,20
2,Arendal,Arendal,Aust-Agder,Agder,Sørlandsk,20
3,Kristiansand,Kristiansand,Vest-Agder,Agder,Sørlandsk,19
4,Songdalen,Kristiansand,Vest-Agder,Agder,Sørlandsk,19


In [36]:
import itertools

for new_kommune in kommune_to_new_fylke.keys():
    if new_kommune not in list(csv_df['new_muni']):
        print(new_kommune)
print('-- Done --')

-- Done --


In [37]:
def get_2024_fylke(row):
    if row.new_muni in kommune_to_new_fylke:
        return kommune_to_new_fylke[row.new_muni]
    else:
        return row.new_county

csv_df['new_county_2024'] = csv_df.apply(lambda x: get_2024_fylke(x), axis=1)
csv_df.head()

,old_muni,new_muni,old_county,new_county,named_dialect,numeric_dialect,new_county_2024
0,Risør,Risør,Aust-Agder,Agder,Sørlandsk,20,Agder
1,Grimstad,Grimstad,Aust-Agder,Agder,Sørlandsk,20,Agder
2,Arendal,Arendal,Aust-Agder,Agder,Sørlandsk,20,Agder
3,Kristiansand,Kristiansand,Vest-Agder,Agder,Sørlandsk,19,Agder
4,Songdalen,Kristiansand,Vest-Agder,Agder,Sørlandsk,19,Agder


In [38]:
csv_df.to_csv('/Users/plparson/Documents/development/DialectMapper/dialect_mapper/mapping_data/muni_county_namedDialect_numericDialect_mapping_manual_additions_renamed_2024.csv', index=False)